# 14장. 반복형, 반복자, 제너레이터

메모리에 다 들어가지 않는 데이터셋을 검색할 때는 한번에 하나씩 필요할 때 가져오도록 구현해야 하며, 이것이 반복자 패턴이 하는 일이다.

파이썬의 컬렉션은 모두 반복형이며, 다음과 같은 연산을 지원하기 위해 내부적으로 반복자를 사용한다.

* for 루프
* 컬렉션형 생성과 확장
* 텍스트 파일을 한 줄씩 반복
* 지능형 리스트/딕셔너리/집합
* 튜플 언패킹
* 함수 호출 시 *를 이용해서 실제 매개변수를 언패킹

이 장에서는 다음과 같은 내용을 다룬다.

* 반복형 객체를 처리하기 위해 내부적으로 `iter()` 내장 함수를 사용하는 방법
* 파이썬에서 고전적인 반복자 패턴을 구현하는 방법
* 제너레이터가 작동하는 방식을 한 줄씩 상세히 설명
* 고전적인 반복자를 제너레이터 함수나 제너레이터 표현식으로 바꾸는 방법
* 표준 라이브러리에서의 범용 제너레이터 함수의 활용
* 제너레이터를 결합하기 위해 새로 추가된 `yield from`을 사용하는 방법
* 사례 연구: 대형 데이터셋을 사용하도록 설계된 데이터베이스 변환 유틸리티에서 제너레이터 함수 이용
* 비슷해 보이는 제너레이터와 코루틴이 실제로는 아주 다르며, 혼합해서 사용하면 안 되는 이유

## 14.1 Sentence 버전 #1: 단어 시퀀스

먼저 Sentence라는 클래스를 구현하면서 반복형을 알아보자.

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+') # \w : 알파벳 및 숫자, + : 연속된 문자열

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # 중복되지 않은 매칭 문자열의 리스트 반환
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 문자열을 축약해서 생성

In [5]:
s = Sentence('"The time has come," the Walrus said,')
s # reprlib.repr() 이 생성한 결과

Sentence('"The time ha... Walrus said,')

In [6]:
for word in s: # Sentence 객체는 반복 가능
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [8]:
s[0], s[5], s[-1] # 인덱스 이용 가능

('The', 'Walrus', 'said')

### 14.1.1 Sequence가 반복 가능한 이유: iter() 함수

파이썬 인터프리터가 x 객체를 반복해야 할 때는 언제나 `iter(x)`를 자동으로 호출한다. `iter()` 내장 함수는 다음 과정을 수행한다.

1. 객체가 `__iter__()` 메서드를 구현하는지 확인하고, 이 메서드를 호출하여 반복자를 가져옴
1. `__iter__()` 메서드가 없고 `__getitem__()`이 있으면 파이썬은 인덱스 0에서부터 시작해서 항목을 순서대로 가져오는 반복자를 생성
1. 이 과정이 모두 실패하면 파이썬은 `TypeError` 오류를 발생시킴

따라서 모든 파이썬 시퀀스는 `__getitem__()`을 구현하고 있으므로 반복할 수 있다(덕 타이핑!). 

구스 타이핑을 이용하면 반복형에 대한 정의가 단순해질 수 있지만, 융퉁성이 떨어진다. 사실 객체를 반복하기 전에 반복형인지 검사하는 건 불필요한데, 이는 바로 `TypeError`를 발생시키기 때문이다. 따라서 `try/except` 블록으로 처리하는 것이 좋다.

## 14.2 반복형과 반복자

* 반복형 : `iter()` 내장 함수가 반복자를 가져올 수 있는 모든 객체(ex:시퀀스)와 반복자를 반환하는 `__iter__()` 메서드를 구현하는 객체를 반복형이라고 한다.
* 반복자 : 다음 항목을 반환하거나, 다음 항목이 없을 때 StopIteration 예외를 발생시키는, 인수를 받지 않는 `__next__()` 메서드를 구현하는 객체. 파이썬 반복자는 `__iter__()` 메서드도 구현하므로 반복형이기도 하다.

파이썬은 반복형 객체에서 반복자를 가져온다. 다음 예제에서 'ABC'는 반복형이다. 반복자가 보이지 않지만 내부 어딘가에 존재한다.

In [9]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


for 문 대신 while 문을 이용하면 다음과 같다.

In [11]:
s = 'ABC'
it = iter(s) # 반복형에서 반복자 it를 생성
while True:
    try:
        print(next(it))
    except StopIteration: # 더이상 항목이 없으면 반복자가 StopIteration 예외 발생시킴
        del it # 반복자 객체 제거
        break

A
B
C


반복자에 대한 표준 인터페이스는 다음과 같은 메서드 두 개를 정의한다.

* `__next__()` : 다음에 사용할 항목 반환. 항목이 없으면 StopIteration 발생
* `__iter__()` : self를 반환.

반복자는 `next()`를 호출하고 StopIteration 예외를 잡는 방법 외에는 항목이 소진되었는지 확인할 방법이 없다. 그리고 반복자는 재설정할 수 없으며, 반복형에 `iter()`를 호출해서 다시 반복자를 생성해야 한다.

In [12]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)
it

In [13]:
next(it)

'Pig'

In [14]:
next(it)

'and'

In [15]:
next(it)

'Pepper'

In [16]:
next(it)

StopIteration: 

In [17]:
list(it)

[]

In [18]:
list(iter(s3))

['Pig', 'and', 'Pepper']

## 14.3 Sentence 버전 #2: 고전적인 반복자

이번에는 고전적인 반복자 패턴에 맞춰 Sentence 클래스를 구현한다. `__iter__()` 특별 메서드를 구현하고 이 메서드가 SentenceIterator를 반환하도록 만든다.

In [19]:
import re
import reprlib

RE_WORD = re.compile('\w+') # \w : 알파벳 및 숫자, + : 연속된 문자열

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # 중복되지 않은 매칭 문자열의 리스트 반환
            
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 문자열을 축약해서 생성
    
    def __iter__(self): # __getitem__() 은 구현하지 않아도 __iter__()가 있으므로 반복형
        return SentenceIterator(self.words)
    
class SentenceIterator:
    
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError: # 단어가 없으면 예외 발생
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self
            

사실 SentenceIterator에서 `__iter__()`를 구현할 필요는 없다. 그렇지만 반복자는 `__next__()`와 `__iter__()` 메서드를 모두 구현해야 `issubclass(SentenceIterator, abc.Iterator)` 테스트를 통과할 수 있다.

### 14.3.1 Sentence를 반복자로 만들기: 좋지 않은 생각

반복형과 반복자를 만들 때 흔히 하는 오류는 둘을 혼동하기 때문에 발생한다. 반복형은 호출될 때마다 반복자를 새로 생성하는 `__iter__()` 메서드를 가지고 있으며, 반복자는 개별 항목을 반환하는 `__next__()` 메서드와 `__iter__()` 메서드를 가지고 있다. 따라서 반복자는 반복형이지만 반복형은 반복자가 아니다.

반복자 패턴은 다음과 같은 용도에 사용해야 한다.

* 집합 객체의 내부 표현을 노출시키지 않고 내용에 접근하는 경우
* 집합 객체의 다중 반복을 지원하는 경우
* 다양한 집합 구조체를 반복하기 위한 통일된 인터페이스를 제공하는 경우


## 14.4 Sentence 버전 #3: 제너레이터 함수

동일한 기능을 파이썬스럽게 구현하려면 SentenceIterator 클래스 대신 제너레이터 함수를 사용한다.

In [20]:
import re
import reprlib

RE_WORD = re.compile('\w+') # \w : 알파벳 및 숫자, + : 연속된 문자열

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # 중복되지 않은 매칭 문자열의 리스트 반환
            
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 문자열을 축약해서 생성
    
    def __iter__(self): 
        for word in self.words:
            yield word # 현재 단어를 생성
        return
    # 별도의 반복자 클래스가 필요 없다

### 14.4.1 제너레이터 함수의 작동 방식